## Invoke w/ boto3

In [1]:
import json 
import boto3 
from tqdm import tqdm
client = boto3.client('runtime.sagemaker')
 
data = {
    "columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}

response = client.invoke_endpoint(EndpointName='mlflow-sean-test',
                                  ContentType='application/json; format=pandas-split',
                                  Body=json.dumps(data))
response_body = response['Body'] 
print(response_body.read().decode('utf-8'))

[5.155298343055017]


## Invoke w/ sigv4

Ref : https://docs.aws.amazon.com/AmazonS3/latest/API/sigv4-auth-using-authorization-header.html  

In [7]:
import sys, os, base64, datetime, hashlib, hmac 
import requests

def sign(key, msg):
    return hmac.new(key, msg.encode("utf-8"), hashlib.sha256).digest()

def getSignatureKey(key, date_stamp, regionName, serviceName):
    kDate = sign(('AWS4' + key).encode('utf-8'), date_stamp)
    kRegion = sign(kDate, regionName)
    kService = sign(kRegion, serviceName)
    kSigning = sign(kService, 'aws4_request')
    return kSigning

method = 'POST'
service = 'sagemaker'

host = 'runtime.sagemaker.ap-northeast-2.amazonaws.com'
region = 'ap-northeast-2'
endpoint = 'https://runtime.sagemaker.ap-northeast-2.amazonaws.com/endpoints/mlflow-sean-test/invocations'
content_type = 'application/x-amz-json-1.0'
request_parameters = json.dumps({
    "columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]})

access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

if access_key is None or secret_key is None:
    print('No access key is available.')
    sys.exit()

t = datetime.datetime.utcnow()
amz_date = t.strftime('%Y%m%dT%H%M%SZ')
date_stamp = t.strftime('%Y%m%d') 

canonical_uri = '/endpoints/mlflow-sean-test/invocations'
canonical_querystring = ''
canonical_headers = 'content-type:' + content_type + '\n' + 'host:' + host + '\n' + 'x-amz-date:' + amz_date + '\n'

signed_headers = 'content-type;host;x-amz-date'
payload_hash = hashlib.sha256(request_parameters.encode('utf-8')).hexdigest()

canonical_request = method + '\n' + canonical_uri + '\n' + canonical_querystring + '\n' + canonical_headers + '\n' + signed_headers + '\n' + payload_hash

algorithm = 'AWS4-HMAC-SHA256'
credential_scope = date_stamp + '/' + region + '/' + service + '/' + 'aws4_request'
string_to_sign = algorithm + '\n' +  amz_date + '\n' +  credential_scope + '\n' +  hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()

signing_key = getSignatureKey(secret_key, date_stamp, region, service)
signature = hmac.new(signing_key, (string_to_sign).encode('utf-8'), hashlib.sha256).hexdigest()

authorization_header = algorithm + ' ' + 'Credential=' + access_key + '/' + credential_scope + ', ' +  'SignedHeaders=' + signed_headers + ', ' + 'Signature=' + signature
headers = {'Content-Type':content_type,
           'X-Amz-Date':amz_date,
           'Authorization':authorization_header}

# ************* SEND THE REQUEST *************
print('\nBEGIN REQUEST++++++++++++++++++++++++++++++++++++')
print('Request URL = ' + endpoint)

r = requests.post(endpoint, data=request_parameters, headers=headers)

print('\nRESPONSE++++++++++++++++++++++++++++++++++++')
print('Response code: %d\n' % r.status_code)
print(r.text)


BEGIN REQUEST++++++++++++++++++++++++++++++++++++
Request URL = https://runtime.sagemaker.ap-northeast-2.amazonaws.com/endpoints/mlflow-sean-test/invocations

RESPONSE++++++++++++++++++++++++++++++++++++
Response code: 403

{"Message":"User: arn:aws:iam::526094755604:user/slayer@mathpresso.com is not authorized to perform: sagemaker:InvokeEndpoint on resource: arn:aws:sagemaker:ap-northeast-2:526094755604:endpoint/mlflow-sean-test with an explicit deny"}
